This Notebook will be used for the initial data exploration phase

In [38]:
import os
import pandas as pd
import pyarrow.parquet as pq
import json

1. Functions

In [39]:
def parse_byte_string(byte_string):
    try:
        decoded_string = byte_string.decode('utf-8')
        json_data = json.loads(decoded_string)
        return json_data
    except Exception as e:
        print(f"Failed to parse: {byte_string}, Error: {e}")
        return None


In [42]:
parquet_dir = '../../../data/netflow_data'  # Adjust this path to where your Parquet files are stored

# Initialize an empty list to store DataFrames
dataframes = []

# Walk through the directory to find all Parquet files
for root, dirs, files in os.walk(parquet_dir):
    for file in files:
        if file.endswith('.parquet'):
            file_path = os.path.join(root, file)
            # Read the Parquet file into a DataFrame
            df = pq.read_table(file_path).to_pandas()
            # Assuming the first column contains byte strings
            first_column_name = df.columns[0]  # Replace with the actual column name if known
            
            # Apply the parsing function to the first column
            df[first_column_name] = df[first_column_name].apply(parse_byte_string)
            
            dataframes.append(df)

# Concatenate all the DataFrames into one DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the first few rows of the combined DataFrame
combined_df.head()

,raw_data,year,month,day
0,"{'as.label': ['PRIVATE', 'PUBLIC'], 'ethernet....",2024,8,25
1,"{'as.label': ['PUBLIC', 'PRIVATE'], 'ethernet....",2024,8,25
2,"{'as.label': ['PUBLIC', 'PRIVATE'], 'ethernet....",2024,8,25
3,"{'as.label': ['PUBLIC', 'PRIVATE'], 'ethernet....",2024,8,25
4,"{'as.label': ['PRIVATE', 'PUBLIC'], 'ethernet....",2024,8,25


In [45]:
combined_df.describe()

,raw_data,year,month,day
count,740,740,740,740
unique,740,1,1,1
top,"{'as.label': ['PRIVATE', 'PUBLIC'], 'ethernet....",2024,8,25
freq,1,740,740,740


In [49]:

json_df = pd.json_normalize(combined_df['raw_data'])


In [51]:
json_df.head()

,as.label,ethernet.ether_type.name,ethernet.header.size,flow.bytes,flow.client.as.asn,flow.client.as.label,flow.client.as.org,flow.client.host.name,flow.client.ip.addr,flow.client.l4.port.id,...,tcp.segs.rst,tcp.segs.syn,tcp.segs.urg,flow.out.netif.index,flow.out.netif.name,flow.out.netif.type.name,icmp.code.name,icmp.type.name,flow.export.proc.pid,flow.meter.packets_drop
0,"[PRIVATE, PUBLIC]",IPv4,14.0,96000.0,0.0,PRIVATE,PRIVATE,192.168.1.2,192.168.1.2,34214.0,...,0.0,66560.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[PUBLIC, PRIVATE]",IPv4,14.0,3328.0,0.0,PRIVATE,PRIVATE,192.168.1.2,192.168.1.2,34214.0,...,0.0,66560.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[PUBLIC, PRIVATE]",IPv4,14.0,3328.0,0.0,PRIVATE,PRIVATE,192.168.1.2,192.168.1.2,34214.0,...,0.0,66560.0,0.0,2319.0,index: 2319,Unknown,NaN,NaN,NaN,NaN
3,"[PUBLIC, PRIVATE]",IPv4,14.0,5376.0,0.0,PRIVATE,PRIVATE,192.168.1.5,192.168.1.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No Code,Echo,NaN,NaN
4,"[PRIVATE, PUBLIC]",IPv4,14.0,5376.0,0.0,PRIVATE,PRIVATE,192.168.1.7,192.168.1.7,NaN,...,NaN,NaN,NaN,2321.0,index: 2321,Unknown,No Code,Echo Reply,NaN,NaN


In [56]:
json_df.columns.to_list()

['as.label',
 'ethernet.ether_type.name',
 'ethernet.header.size',
 'flow.bytes',
 'flow.client.as.asn',
 'flow.client.as.label',
 'flow.client.as.org',
 'flow.client.host.name',
 'flow.client.ip.addr',
 'flow.client.l4.port.id',
 'flow.client.l4.port.name',
 'flow.collect.timestamp',
 'flow.community.id',
 'flow.conversation.id',
 'flow.direction.name',
 'flow.dst.as.asn',
 'flow.dst.as.label',
 'flow.dst.as.org',
 'flow.dst.host.name',
 'flow.dst.ip.addr',
 'flow.dst.l4.port.id',
 'flow.dst.l4.port.name',
 'flow.end.reason.name',
 'flow.end.time_offset',
 'flow.export.host.name',
 'flow.export.ip.addr',
 'flow.export.l4.port.id',
 'flow.export.timestamp',
 'flow.export.type',
 'flow.export.version.name',
 'flow.export.version.ver',
 'flow.in.bytes',
 'flow.in.bytes_sumsqrs',
 'flow.in.bytes_total',
 'flow.in.bytes_total_sumsqrs',
 'flow.in.dst.mac.addr',
 'flow.in.ip.dscp.name',
 'flow.in.ip.ecn.name',
 'flow.in.ip.prec.name',
 'flow.in.l2.bytes',
 'flow.in.l2.bytes_total',
 'flow.in